# Tarea M8 MPAD de Mouzo Monteagudo_Lucio Alejandro

#### Importar pandas y las librerías utilizadas para resolver la tarea.

In [10]:
# Importe todas las librerias que se usan en esta notebook
import os
import requests
import pandas as pd
import time
import json


# 1.	Elija una API deportiva o un conjunto de datos deportivos de las siguientes opciones: 


    * API abierta de datos deportivos
    * API de una página web deportiva
    * Conjunto de datos de StatsBomb Open Data
    * Simulador de API como base de datos (si no tiene acceso a una API real)


In [11]:
# Cuente y describa que api eligió

La API que he escogido es la de Football-Data, que ofrece datos acerca de las competiciones. Tiene una parte gratuita con una cantidad de competiciones, y luego varias secciones de pago donde se puede incrementar la cantidad de estadísticas y datos, y también la cantidad de competiciones que podemos visualizar.
Esta API, tiene una cantidad límite de solicitudes dependiendo de la suscripción que se tenga, y también se necesita entrar a través de una clave.
El formato de los datos que nos proporciona es en archivo JSON.

# 2.	Realice las siguientes tareas, creando funciones para cada paso principal: 

### a) Análisis y Planificación

* Analice la estructura y documentación de la API o conjunto de datos elegido.  
* Identifique los endpoints relevantes o los datos específicos a extraer. 
* Planifique la estrategia de adquisición de datos, considerando límites de tasa si es aplicable. 
* Documente las consideraciones éticas y legales del uso de la API o datos. 


In [12]:
# Describa los endpoint que utilizara en el proceso.

Los puntos en los que nos hemos centraaado para recoger la información han sido los endpoints de las competiciones y de las clasificaciones de cada una de las ligas.
Para la autenticación de los datos se necesita el X_Auth_token, con la clave que nosha proporcionado al loogearnos, y también he añadido puntos en los que se avise en caso de error para ir directo y manejarlos. 
Para los límites de tasas he añadido un tiempo entre cada una de las solicitudes para no sobrecargar la página y no se pase el límite permitido por la página por minuto.

### b) Implementación de la Adquisición de Datos  
* Desarrolle scripts para interactuar con la API o cargar los datos utilizando las bibliotecas apropiadas (requests, pandas, etc.).  
* Implemente manejo de autenticación si es necesario. 
* Incluya manejo de errores y reintentos. 
* Asegúrese de respetar los límites de tasa y políticas de uso de la API. 


In [13]:
# Genere la implementación de adquisición de datos mediante la API

In [14]:
import os
import requests
import pandas as pd
import time

# Obtener la API Key de las variables de entorno o ingresarla manualmente
api_key = os.getenv("FOOTBALL_DATA_API_KEY", "2b4bfa139edd4579bb1f791da1bcc3b4")

# Headers con tu API Key
headers = {
    "X-Auth-Token": api_key
}

# Función para obtener la lista de todas las ligas disponibles
def obtener_ligas(headers):
    url = "https://api.football-data.org/v4/competitions"
    response = requests.get(url, headers=headers, timeout=10)
    if response.status_code == 200:
        return response.json()['competitions']
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Función para obtener la clasificación de una liga con manejo de errores y reintentos
def obtener_clasificacion_liga(league_id, headers, retries=3):
    url = f"https://api.football-data.org/v4/competitions/{league_id}/standings"
    
    # Reintentos en caso de fallo
    for intento in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                standings_data = response.json()
                standings = standings_data['standings'][0]['table']
                return pd.DataFrame([{
                    'Equipo': team['team']['name'],
                    'Liga': standings_data['competition']['name'],
                    'Posición': team['position'],
                    'Puntos': team['points'],
                    'Partidos Jugados': team['playedGames'],
                    'Ganados': team['won'],
                    'Empatados': team['draw'],
                    'Perdidos': team['lost'],
                    'Goles a Favor': team['goalsFor'],
                    'Goles en Contra': team['goalsAgainst'],
                    'Diferencia de Goles': team['goalDifference']
                } for team in standings])
            else:
                print(f"Error {response.status_code}: {response.text}")
        except requests.exceptions.RequestException as e:
            print(f"Error en la solicitud: {e}")
            print(f"Reintento {intento+1}/{retries}...")
            time.sleep(2)  # Pausa antes de reintentar
    return None

# Función para obtener y combinar clasificaciones de todas las ligas disponibles
def obtener_clasificaciones_todas_ligas(headers):
    ligas = obtener_ligas(headers)
    clasificaciones = pd.DataFrame()
    
    for liga in ligas:
        liga_id = liga['id']
        clasificacion = obtener_clasificacion_liga(liga_id, headers)
        if clasificacion is not None:
            clasificaciones = pd.concat([clasificaciones, clasificacion], ignore_index=True)
        else:
            print(f"No se pudieron obtener los datos para la liga {liga['name']}")
        
        # Pausar entre solicitudes para respetar los límites de tasa de la API
        time.sleep(5)
    
    return clasificaciones

# Obtener las clasificaciones de todas las ligas
clasificaciones = obtener_clasificaciones_todas_ligas(headers)

# Mostrar los datos combinados
print("Clasificación de todas las ligas:")
print(clasificaciones)

# Guardar los datos en un archivo CSV
clasificaciones.to_csv('clasificacion_todas_ligas.csv', index=False)
print("Datos de clasificación de todas las ligas guardados en 'clasificacion_todas_ligas.csv'.")


Error 429: {"message":"You reached your request limit. Wait 7 seconds.","errorCode":429}
Error 429: {"message":"You reached your request limit. Wait 7 seconds.","errorCode":429}
Error 429: {"message":"You reached your request limit. Wait 7 seconds.","errorCode":429}
No se pudieron obtener los datos para la liga Copa Libertadores
Error 429: {"message":"You reached your request limit. Wait 1 seconds.","errorCode":429}
Error 429: {"message":"You reached your request limit. Wait 1 seconds.","errorCode":429}
Error 429: {"message":"You reached your request limit. Wait 1 seconds.","errorCode":429}
No se pudieron obtener los datos para la liga Primera Division
Clasificación de todas las ligas:
               Equipo                           Liga  Posición  Puntos  \
0         Botafogo FR  Campeonato Brasileiro Série A         1      79   
1        SE Palmeiras  Campeonato Brasileiro Série A         2      73   
2         CR Flamengo  Campeonato Brasileiro Série A         3      70   
3        

In [18]:
df['Liga'].unique()

array(['Campeonato Brasileiro Série A', 'Championship', 'Premier League',
       'UEFA Champions League', 'European Championship', 'Ligue 1',
       'Bundesliga', 'Serie A', 'Eredivisie', 'Primeira Liga',
       'FIFA World Cup'], dtype=object)

Para la primera parte en la que recogeremos las distintas clasificaciones que se encuentran en la API. Para ello inicialmente marcamos un GET con la clave que tengo de usuario.
Seguimos con la definición de la función para obtener las ligas, donde haremos un bucle por las distintas ligas en las que iremos concatenando la clasificación a las anteriores, con la posibilidad de que si no las encuentre nos de error.
de esta forma si todo va bien ya tendremos un Data Frame con todas las ligas


* Limpie los datos adquiridos (manejo de valores faltantes, corrección de formatos, etc.). 
* Realice transformaciones necesarias para el análisis (creación de nuevas columnas, agregaciones, etc.). 
* Guarde los datos procesados en un formato estructurado (CSV, JSON, etc.). 


Lo que hemos realizado es un filtro para quedarnos únicamente con las ligas que son de nuestro interés, que en este caso son las cinco grandes ligas.

In [15]:
df=pd.read_csv('clasificacion_todas_ligas.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Equipo               200 non-null    object
 1   Liga                 200 non-null    object
 2   Posición             200 non-null    int64 
 3   Puntos               200 non-null    int64 
 4   Partidos Jugados     200 non-null    int64 
 5   Ganados              200 non-null    int64 
 6   Empatados            200 non-null    int64 
 7   Perdidos             200 non-null    int64 
 8   Goles a Favor        200 non-null    int64 
 9   Goles en Contra      200 non-null    int64 
 10  Diferencia de Goles  200 non-null    int64 
dtypes: int64(9), object(2)
memory usage: 17.3+ KB


Podemos observar ek tipo de nuestras columnas, en las que no tenemos ningún elemento nulo, pero además solo son de tipo objeto el equipo y la liga, y las otras son entreros.

In [16]:
# Crear la columna de Efectividad (%)
df.loc[:, 'Efectividad (%)'] = (df['Puntos'] / (df['Partidos Jugados'] * 3)) * 100

# Crear la columna de Promedio de Goles por Partido
df.loc[:, 'Promedio Goles a Favor'] = df['Goles a Favor'] / df['Partidos Jugados']

# Crear la columna de Promedio de Goles en Contra por Partido
df.loc[:, 'Promedio Goles en Contra'] = df['Goles en Contra'] / df['Partidos Jugados']

# Crear la columna de Diferencia de Goles por Partido
df.loc[:, 'Diferencia de Goles por Partido'] = df['Diferencia de Goles'] / df['Partidos Jugados']

# Mostrar un resumen de las nuevas columnas creadas
print("Datos con las nuevas columnas añadidas:")
print(df[['Equipo', 'Liga', 'Puntos', 'Efectividad (%)', 'Promedio Goles a Favor', 'Promedio Goles en Contra', 'Diferencia de Goles por Partido']])

Datos con las nuevas columnas añadidas:
               Equipo                           Liga  Puntos  Efectividad (%)  \
0         Botafogo FR  Campeonato Brasileiro Série A      79        69.298246   
1        SE Palmeiras  Campeonato Brasileiro Série A      73        64.035088   
2         CR Flamengo  Campeonato Brasileiro Série A      70        61.403509   
3        Fortaleza EC  Campeonato Brasileiro Série A      68        59.649123   
4    SC Internacional  Campeonato Brasileiro Série A      65        57.017544   
..                ...                            ...     ...              ...   
195         FC Arouca                  Primeira Liga       8        20.512821   
196       Netherlands                 FIFA World Cup       7        77.777778   
197           Senegal                 FIFA World Cup       6        66.666667   
198           Ecuador                 FIFA World Cup       4        44.444444   
199             Qatar                 FIFA World Cup       0         

Lo que hemos hecho ha sido crear 4 nuevas columnas, en las que encontramos la efectividad de puntos, el promedio de goles tanto a favor como en contra por partido y la diferencia de goles por partido, que las añadimos a las columnas que ya teníamos.

In [17]:
# Guardar el DataFrame en un archivo CSV
df.to_csv('clasificacion_ligas_procesadas.csv', index=False)
print("Datos guardados en 'clasificacion_cinco_ligas_procesadas.csv'.")

Datos guardados en 'clasificacion_cinco_ligas_procesadas.csv'.


Lo último que haremos en este apartado será guardar el DataFrame en archivos .csv